# Модуль 1. Контроль торговых точек по среднему чеку
**Задача:** Сеть супермаркетов, 150+ торговых точек. Еженедельно нужно выявлять точки с аномально низким средним чеком, отсекая те, где мало данных для статистической значимости.

In [ ]:
import pandas as pd

In [ ]:
# Загружаем данные
df = pd.read_excel('Лямбда-функции. Список транзакций.xlsx')
df.head()

In [ ]:
# Средний чек по сети — эталон для сравнения
avg_check_network = df['check'].mean()
print(f'Средний чек по сети: {avg_check_network:.2f}')

In [ ]:
# Группировка по торговым точкам
points_stats = df.groupby('id_point').agg(
    avg_check=('check', 'mean'),
    transactions=('check', 'count')
).round(2).reset_index()
points_stats.head()

In [ ]:
def check_trading_point(avg_check, transactions, 
                       threshold_avg, threshold_cnt, 
                       deviation_percent=0.9):
    """
    Проверка торговой точки на аномалию среднего чека.
    
    Параметры:
        avg_check: средний чек точки
        transactions: количество транзакций
        threshold_avg: целевой средний чек (обычно среднее по сети)
        threshold_cnt: минимальное количество транзакций для доверия
        deviation_percent: допустимое отклонение (0.9 = 90%)
    
    Возвращает:
        'ok' — точка в норме
        'low_check' — низкий средний чек
        'low_data' — мало транзакций, нет доверия к среднему
    """
    try:
        if transactions < threshold_cnt:
            return 'low_data'
        if avg_check < threshold_avg * deviation_percent:
            return 'low_check'
        return 'ok'
    except TypeError:
        return 'error_data'

In [ ]:
# Применяем проверку ко всем точкам
points_stats['check_result'] = points_stats.apply(
    lambda row: check_trading_point(
        row['avg_check'],
        row['transactions'],
        avg_check_network,
        300,  # порог транзакций
        0.9
    ), axis=1
)

In [ ]:
# Смотрим результаты
points_stats['check_result'].value_counts()

In [ ]:
# Доля точек, прошедших проверку
ok_share = len(points_stats[points_stats['check_result'] == 'ok']) / len(points_stats)
print(f'Доля точек в норме: {ok_share:.1%}')
print(f'Требуется контроль: {len(points_stats[points_stats["check_result"] == "low_check"])} точек')
print(f'Мало данных: {len(points_stats[points_stats["check_result"] == "low_data"])} точек')

In [ ]:
# Выгрузка для отдела контроля
control_list = points_stats[points_stats['check_result'].isin(['low_check', 'low_data'])].copy()
control_list = control_list.sort_values('avg_check')
control_list.head(10)